In [7]:
import csv

from stemmers import stemmers
from create import tokenize_values

In [8]:
fname = "../dict/mfd2.0.dic"  # Hopp
# fname = "../dict/mft_original.dic"  # multilabel
# fname = "../dict/Provisional_dictionary.dic"
# fname = "../dict/Refined_dictionary.dic" # Ponizovskiy

In [9]:
# phases: 0 - init; 1 - classes; 2 - vocab
phases = 0
classes = {}
codes = {}

with open(fname) as fin:
    src = csv.reader(fin, delimiter="\t")
    for r in src:
        if not r:
            continue
        if r[0] == "%":
            phases += 1
            continue
        if phases == 1:
            classes[r[0]] = r[1]
            codes[r[0]] = []
            continue
        assert phases == 2, f"Not expected to arrive to phase {phases}"
        codes[r[1]] += [r[0]]
classes

{'1': 'care.virtue',
 '2': 'care.vice',
 '3': 'fairness.virtue',
 '4': 'fairness.vice',
 '5': 'loyalty.virtue',
 '6': 'loyalty.vice',
 '7': 'authority.virtue',
 '8': 'authority.vice',
 '9': 'sanctity.virtue',
 '10': 'sanctity.vice'}

In [10]:
for s in stemmers.keys():
    backref = {}
    for c, r in codes.items():
        for v in r:
            stem = stemmers[s](v)
            if stem in backref:
                backref[stem] |= set([c])
            else:
                backref[stem] = set([c])
    fname_out = fname.replace(".dic", f".{s}.csv")
    with open(fname_out, "w") as fout:
        dest = csv.writer(fout)
        for c, r in codes.items():
            row = [classes[c]] + [v for v in r if len(backref[stemmers[s](v)]) == 1]
            dest.writerow(row)

Loaded 4225 morphemens
